In [ ]:
import os
import pathlib
import subprocess
import sys
import typing

import networkx as nx
import yaml

In [ ]:
EXECUTABLE_PATH = {
    "cvc": "...",
    "vc": "...",
    "local_ratio_vc": "...",
}

data_dir = pathlib.Path("data/")
results_dir = pathlib.Path("results/")
results_dir.mkdir(exist_ok=True)

In [ ]:
def graph_in_dir_is_connected(dir: pathlib.Path) -> bool:
    with open(dir / "properties.yaml") as f:
        props = yaml.load(f, yaml.SafeLoader)
        return props["connected"]

In [ ]:
def run_cvc(g: nx.Graph) -> tuple[str, str]:
    return run_executable_on_graph(EXECUTABLE_PATH["cvc"], g)


def run_vc(g: nx.Graph) -> tuple[str, str]:
    return run_executable_on_graph(EXECUTABLE_PATH["vc"], g)


def run_local_ratio_vc(g: nx.Graph) -> tuple[str, str]:
    return run_executable_on_graph(EXECUTABLE_PATH["local_ratio_vc"], g)


def run_executable_on_graph(exe_path: pathlib.Path | os.PathLike[typing.Any] | str, g: nx.Graph) -> tuple[str, str]:
    assert pathlib.Path(exe_path).is_file()

    proc = subprocess.run(
        str(exe_path),
        input=nx.generate_edgelist(g, data=False),
        capture_output=True,
        text=True,
    )

    assert proc.returncode == 0, f'error: subprecess process returned {proc.returncode}; stderr= "{proc.stderr}"'
    nodes, size = proc.stdout.split(" => ", maxsplit=1)
    return nodes.strip(), size.strip()

In [ ]:
for dir in data_dir.iterdir():
    print(f"{dir.name} => ", end="")

    if not graph_in_dir_is_connected(dir):
        print("skipeed (not connected)")
        continue

    g = nx.read_edgelist(dir / "graph.edges")

    cvc_nodes, cvc_card = run_cvc(g)
    vc_nodes, vc_card = run_vc(g)
    local_ratio_vc_nodes, local_ratio_vc_card = run_local_ratio_vc(g)

    res_dir = results_dir / dir.name
    res_dir.mkdir(exist_ok=True)

    for contents, filename in [
        (cvc_nodes, "cvc_nodes.txt"),
        (cvc_card, "cvc_cardinality.txt"),
        (vc_nodes, "vc_nodes.txt"),
        (vc_card, "vc_cardinality.txt"),
        (local_ratio_vc_nodes, "local_ratio_vc_nodes.txt"),
        (local_ratio_vc_card, "local_ratio_vc_cardinality.txt"),
    ]:
        with open(res_dir / filename, "w") as f:
            _ = f.write(contents)

    print(f"results saved to {results_dir.name}/{dir.name}")